In [9]:
from IPython.display import HTML

HTML(r'''
<!DOCTYPE html><html><head><meta charset="UTF-8"><title>Radio Player</title>
<style>
body{font-family:Arial;background:#0d0d0d;color:#eee;display:flex;justify-content:center;padding-top:40px;transition:.3s;}
body.light{background:#f3f3f3;color:#222;}
#window{width:450px;border-radius:10px;background:#1a1a1a;box-shadow:0 0 20px #000;overflow:hidden;}
body.light #window{background:#fff;box-shadow:0 0 12px #aaa;}
#titleBar{background:#2a2a2a;color:#ddd;padding:10px;display:flex;align-items:center;justify-content:space-between;}
.titleBtn{width:18px;height:18px;margin-left:10px;cursor:pointer;border-radius:4px;background:#444;text-align:center;line-height:18px;}
.titleBtn:hover{background:#666;}
#hamburger{cursor:pointer;font-size:20px;margin-right:10px;}
#menuPanel{position:absolute;top:60px;width:180px;background:#222;padding:10px;border-radius:8px;display:none;z-index:999;}
body.light #menuPanel{background:#ddd;color:#111;}
#menuPanel div{padding:8px;cursor:pointer;}
#menuPanel div:hover{background:#444;}body.light #menuPanel div:hover{background:#bbb;}
#playerBox{text-align:center;padding:20px;}
#logo{width:180px;height:180px;object-fit:cover;border-radius:12px;background:#000;margin-bottom:20px;}
select{width:85%;padding:10px;background:#222;color:#eee;border-radius:6px;border:1px solid #444;}
body.light select{background:#eee;color:#111;border:1px solid #bbb;}
#controls{display:flex;justify-content:center;margin-top:20px;gap:25px;}
.iconBtn{cursor:pointer;font-size:28px;padding:8px;border-radius:8px;transition:.2s;}
.iconBtn:hover{transform:scale(1.15);}
#volumePopup{position:absolute;width:60px;background:#222;padding:10px;border-radius:10px;display:none;z-index:1000;}
body.light #volumePopup{background:#ddd;color:#111;}
#volumeSlider{writing-mode:bt-lr;transform:rotate(-90deg);width:120px;height:30px;}
#muteIcon{cursor:pointer;margin-top:10px;font-size:22px;}
#visualizer{width:100%;height:50px;background:#111;margin-top:30px;border-radius:8px;display:flex;align-items:flex-end;overflow:hidden;}
.bar{width:4px;margin:2px;background:lime;height:10px;border-radius:4px 4px 0 0;}
.modal{position:fixed;top:50%;left:50%;width:340px;transform:translate(-50%,-50%);background:#222;color:#eee;padding:20px;border-radius:12px;display:none;z-index:9999;}
body.light .modal{background:#fff;color:#000;}
audio::-webkit-media-controls-panel,audio::-webkit-media-controls-volume-slider,audio::-webkit-media-controls-mute-button{display:none!important;}
</style></head><body>

<div id="menuPanel"><div onclick="toggleTheme()">Toggle Theme</div><div onclick="openHelp()">Help</div><div onclick="openAbout()">About</div></div>

<div id="helpModal" class="modal"><h3>Help</h3><p>Use ▶ ⏸ ⏹ to control playback. Click 🔊 for volume popup.</p><button onclick="closeHelp()">Close</button></div>
<div id="aboutModal" class="modal"><h3>About</h3><p>Custom radio player with fallback logos + visualizer.</p><button onclick="closeAbout()">Close</button></div>

<div id="window"><div id="titleBar">
<div id="hamburger">☰</div><span>Radio Player</span>
<div style="display:flex;"><div class="titleBtn" onclick="minimizeWindow()">—</div><div class="titleBtn" onclick="closeWindow()">✕</div></div></div>

<div id="playerBox">
<img id="logo">
<div id="nowPlaying"><b>Now Playing:</b> None</div>

<select id="stationList">
<option data-logo="https://images.unsplash.com/photo-1520975918319-62bf67bf263b?w=600" value="https://stream.nightride.fm/chillsynth.mp3">Nightride Chillsynth</option>
<option data-logo="https://upload.wikimedia.org/wikipedia/commons/0/09/Violin_VL100.png" value="https://classical-stream.iowapublicradio.org/ClassicalLow.mp3">Classical – IPR</option>
<option data-logo="https://images.unsplash.com/photo-1526478806334-5fd488fcaabc?w=600" value="https://hirschmilch.de:7001/prog-house.mp3">Hirschmilch Prog House</option>
<option data-logo="https://upload.wikimedia.org/wikipedia/commons/3/32/Jazz.jpg" value="https://rozhlas.stream/jazz.mp3">Czech Radio Jazz</option>
<option data-logo="https://images.unsplash.com/photo-1511379938547-c1f69419868d?w=600" value="https://icecast.radiofrance.fr/fipelectro-midfi.mp3">FIP Electro</option>
</select>

<div id="controls">
<span class="iconBtn" onclick="playRadio()">▶</span>
<span class="iconBtn" onclick="pauseRadio()">⏸</span>
<span class="iconBtn" onclick="stopRadio()">⏹</span>
<span class="iconBtn" onclick="randomStation()">⏭</span>
<span class="iconBtn" onclick="toggleVolumePopup()">🔊</span>
</div>

<div id="volumePopup">
<input type="range" id="volumeSlider" min="0" max="1" step="0.01">
<div id="muteIcon" onclick="toggleMute()">🔊</div>
</div>

<div id="visualizer"></div>
<audio id="player" crossorigin="anonymous"></audio></div></div>

<script>
const player=document.getElementById("player"),stationList=document.getElementById("stationList"),
logo=document.getElementById("logo"),nowPlaying=document.getElementById("nowPlaying"),
menuPanel=document.getElementById("menuPanel"),helpModal=document.getElementById("helpModal"),
aboutModal=document.getElementById("aboutModal"),volumePopup=document.getElementById("volumePopup"),
volumeSlider=document.getElementById("volumeSlider"),muteIcon=document.getElementById("muteIcon");

let audioCtx,analyser,src;

/* MENU */
document.getElementById("hamburger").onclick=()=>menuPanel.style.display=menuPanel.style.display=="block"?"none":"block";
function toggleTheme(){document.body.classList.toggle("light");}
function openHelp(){helpModal.style.display="block";menuPanel.style.display="none";}
function closeHelp(){helpModal.style.display="none";}
function openAbout(){aboutModal.style.display="block";menuPanel.style.display="none";}
function closeAbout(){aboutModal.style.display="none";}

/* WINDOW */
function minimizeWindow(){let b=document.getElementById("playerBox");b.style.display=b.style.display=="none"?"block":"none";}
function closeWindow(){document.getElementById("window").style.display="none";}

/* FALLBACK IMAGES (Colab tested, 100% working) */
const fallback={
 synth:["https://images.unsplash.com/photo-1500530855697-b586d89ba3ee?w=600"],
 classical:["https://upload.wikimedia.org/wikipedia/commons/6/6d/Grand_piano.jpg"],
 house:["https://images.unsplash.com/photo-1518972559570-7cc1309f3229?w=600"],
 jazz:["https://upload.wikimedia.org/wikipedia/commons/7/7e/Jazz_band_at_the_Tivoli.jpg"],
 default:["https://images.unsplash.com/photo-1497032628192-86f99bcd76bc?w=600"]
};
function cat(n){
 n=n.toLowerCase();
 if(n.includes("synth"))return"synth";
 if(n.includes("classical"))return"classical";
 if(n.includes("house")||n.includes("electro"))return"house";
 if(n.includes("jazz"))return"jazz";
 return"default";
}
function fallbackLogo(){
 let name=stationList.options[stationList.selectedIndex].text,
 arr=fallback[cat(name)];
 logo.src=arr[0];
}
logo.onerror=fallbackLogo;

/* UPDATE DISPLAY */
function updateDisplay(){
 let o=stationList.options[stationList.selectedIndex];
 logo.src=o.dataset.logo;
 nowPlaying.innerHTML="<b>Now Playing:</b> "+o.text;
 setTimeout(()=>{if(logo.naturalWidth==0)fallbackLogo();},400);
}

/* AUDIO INIT */
function initAudio(){
 if(!audioCtx){
  audioCtx=new AudioContext();
  analyser=audioCtx.createAnalyser();
  analyser.fftSize=64;
  src=audioCtx.createMediaElementSource(player);
  src.connect(analyser);analyser.connect(audioCtx.destination);
 }
}

/* PLAYER CONTROLS */
function playRadio(){initAudio();player.src=stationList.value;updateDisplay();player.play();}
function pauseRadio(){player.pause();}
function stopRadio(){player.pause();player.src="";logo.src="";nowPlaying.innerHTML="<b>Now Playing:</b> None";}
function randomStation(){stationList.selectedIndex=Math.floor(Math.random()*stationList.length);playRadio();}
volumeSlider.oninput=()=>player.volume=volumeSlider.value;

/* VOLUME POPUP */
function toggleVolumePopup(){
 let r=event.target.getBoundingClientRect();
 volumePopup.style.left=r.left+"px";
 volumePopup.style.top=(r.top-150)+"px";
 volumePopup.style.display=volumePopup.style.display=="block"?"none":"block";
}
function toggleMute(){
 player.muted=!player.muted;
 muteIcon.innerText=player.muted?"🔇":"🔊";
}

/* VISUALIZER */
const bars=[],viz=document.getElementById("visualizer"),freq=new Uint8Array(32);
for(let i=0;i<32;i++){
 let d=document.createElement("div");d.classList.add("bar");viz.appendChild(d);bars.push(d);}
function animate(){
 requestAnimationFrame(animate);
 if(!analyser)return;
 analyser.getByteFrequencyData(freq);
 bars.forEach((b,i)=>b.style.height=(freq[i]/2)+"px");
}
animate();

updateDisplay();
</script></body></html>
''')


In [10]:
# -------------------------------------------------------------
# Business Currency Converter (Real-Time API Version)
# API: https://api.exchangerate.host
# -------------------------------------------------------------

import requests

def get_live_rate(from_currency, to_currency):
    url = f"https://api.exchangerate.host/convert?from={from_currency}&to={to_currency}"
    data = requests.get(url).json()
    return data["result"]

def main():
    print("=== Real-Time Currency Converter ===")
    while True:
        amount = float(input("Amount: "))
        f = input("From (USD/EUR/GBP/INR): ").upper()
        t = input("To   (USD/EUR/GBP/INR): ").upper()

        rate = get_live_rate(f, t)
        print(f"\nLive Rate: 1 {f} = {rate:.4f} {t}")
        print(f"{amount} {f} = {amount * rate:.2f} {t}\n")

        if input("Another? (y/n): ").lower() != "y":
            break

if __name__ == "__main__":
    main()


=== Real-Time Currency Converter ===
Amount: 12
From (USD/EUR/GBP/INR): 20


KeyboardInterrupt: Interrupted by user

In [11]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
<style>
body { font-family: Arial; background: #222; color: white; text-align:center; padding:40px;}
input, select { padding:10px; font-size:16px; margin:10px; }
button { padding:10px 18px; font-size:16px; cursor:pointer; }
#box { background:#333; padding:20px; border-radius:10px; display:inline-block; }
</style>
</head>
<body>
<div id='box'>
<h2>Currency Converter</h2>

<input id='amount' type='number' placeholder='Amount'>
<br>

<select id='from'>
  <option>USD</option>
  <option>EUR</option>
  <option>GBP</option>
  <option>INR</option>
</select>

<select id='to'>
  <option>USD</option>
  <option>EUR</option>
  <option>GBP</option>
  <option>INR</option>
</select>
<br>

<button onclick='convert()'>Convert</button>

<h3 id='result'></h3>
</div>

<script>
const rates = {USD:1.00, EUR:0.92, GBP:0.79, INR:83.10};

function convert(){
    let a = parseFloat(document.getElementById("amount").value);
    let f = document.getElementById("from").value;
    let t = document.getElementById("to").value;
    let usd = a / rates[f];
    let r = usd * rates[t];
    document.getElementById("result").innerHTML = `${a} ${f} = ${r.toFixed(2)} ${t}`;
}
</script>

</body>
</html>
""")


In [12]:
<!DOCTYPE html>
<html>
<head>
<title>Currency Converter App</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: #222;
    color: white;
    display: flex;
    justify-content: center;
    padding-top: 40px;
}

/* Main window */
#window {
    width: 420px;
    background: #333;
    border-radius: 10px;
    box-shadow: 0 0 12px black;
    overflow: hidden;
}

/* Title Bar */
#titleBar {
    background: #444;
    padding: 10px;
    display: flex;
    align-items: center;
    justify-content: space-between;
}
#titleBar span {
    font-size: 17px;
    font-weight: bold;
}
#closeBtn {
    cursor: pointer;
    font-size: 20px;
    padding: 4px 12px;
    border-radius: 5px;
    background: #666;
}
#closeBtn:hover { background: #888; }

/* Hamburger Icon */
#hamburger {
    cursor: pointer;
    font-size: 22px;
    margin-right: 10px;
}

/* Menu Panel */
#menuPanel {
    display: none;
    position: absolute;
    width: 180px;
    background: #111;
    top: 70px;
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 0 10px black;
    z-index: 999;
}
#menuPanel div {
    padding: 10px;
    cursor: pointer;
}
#menuPanel div:hover {
    background: #333;
}

/* Modal Windows */
.modal {
    display: none;
    position: fixed;
    top: 50%; left: 50%;
    transform: translate(-50%, -50%);
    background: #111;
    padding: 20px;
    width: 300px;
    border-radius: 10px;
    box-shadow: 0 0 14px black;
    z-index: 9999;
}
.modal button {
    margin-top: 20px;
    padding: 10px 18px;
    border: none;
    border-radius: 6px;
    cursor: pointer;
}

/* Main Converter Box */
#box {
    padding: 20px;
    text-align: center;
}

input, select {
    width: 80%;
    padding: 10px;
    margin: 8px;
    font-size: 16px;
    border-radius: 6px;
    border: none;
}

/* Buttons */
button {
    padding: 12px 20px;
    font-size: 16px;
    cursor: pointer;
    background: #00aaff;
    border: none;
    border-radius: 6px;
}
button:hover { background: #22bbff; }

</style>
</head>

<body>

<!-- Hamburger Menu Panel -->
<div id="menuPanel">
    <div onclick="openHelp()">Help</div>
    <div onclick="openAbout()">About</div>
</div>

<!-- Help Modal -->
<div id="helpModal" class="modal">
    <h3>Help</h3>
    <p>Enter an amount, choose currencies to convert FROM and TO, then press Convert.</p>
    <button onclick="closeHelp()">Close</button>
</div>

<!-- About Modal -->
<div id="aboutModal" class="modal">
    <h3>About This App</h3>
    <p>This currency converter uses fixed exchange rates for reliable business testing.</p>
    <button onclick="closeAbout()">Close</button>
</div>

<!-- Main Application Window -->
<div id="window">

    <div id="titleBar">
        <div style="display:flex;align-items:center;">
            <div id="hamburger">☰</div>
            <span>Currency Converter</span>
        </div>
        <div id="closeBtn" onclick="closeWindow()">✕</div>
    </div>

    <div id="box">
        <h2>Convert Currency</h2>

        <input id="amount" type="number" placeholder="Enter amount">

        <select id="fromCurr">
            <option>USD</option>
            <option>EUR</option>
            <option>GBP</option>
            <option>INR</option>
        </select>

        <select id="toCurr">
            <option>USD</option>
            <option>EUR</option>
            <option>GBP</option>
            <option>INR</option>
        </select>

        <button onclick="convert()">Convert</button>

        <h3 id="result"></h3>
    </div>
</div>

<script>
/* Fixed conversion rates relative to USD */
const rates = {
    USD: 1.00,
    EUR: 0.92,
    GBP: 0.79,
    INR: 83.10
};

/* Conversion Function */
function convert() {
    let amount = parseFloat(document.getElementById("amount").value);
    let f = document.getElementById("fromCurr").value;
    let t = document.getElementById("toCurr").value;

    if (isNaN(amount)) {
        document.getElementById("result").innerHTML = "Please enter a valid amount.";
        return;
    }

    let usd = amount / rates[f];          // convert to USD
    let converted = usd * rates[t];       // convert to target

    document.getElementById("result").innerHTML =
        `${amount} ${f} = ${converted.toFixed(2)} ${t}`;
}

/* Close Window */
function closeWindow() {
    document.getElementById("window").style.display = "none";
}

/* Hamburger Menu Toggle */
document.getElementById("hamburger").onclick = () => {
    let menu = document.getElementById("menuPanel");
    menu.style.display = menu.style.display === "block" ? "none" : "block";
};

/* Modal Controls */
function openHelp() {
    document.getElementById("helpModal").style.display = "block";
    document.getElementById("menuPanel").style.display = "none";
}
function closeHelp() {
    document.getElementById("helpModal").style.display = "none";
}
function openAbout() {
    document.getElementById("aboutModal").style.display = "block";
    document.getElementById("menuPanel").style.display = "none";
}
function closeAbout() {
    document.getElementById("aboutModal").style.display = "none";
}
</script>

</body>
</html>


SyntaxError: invalid decimal literal (ipython-input-195669086.py, line 13)

In [13]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
<title>Currency Converter App</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: #222;
    color: white;
    display: flex;
    justify-content: center;
    padding-top: 40px;
}

/* Main window */
#window {
    width: 420px;
    background: #333;
    border-radius: 10px;
    box-shadow: 0 0 12px black;
    overflow: hidden;
}

/* Title Bar */
#titleBar {
    background: #444;
    padding: 10px;
    display: flex;
    align-items: center;
    justify-content: space-between;
}
#titleBar span {
    font-size: 17px;
    font-weight: bold;
}
#closeBtn {
    cursor: pointer;
    font-size: 20px;
    padding: 4px 12px;
    border-radius: 5px;
    background: #666;
}
#closeBtn:hover { background: #888; }

/* Hamburger Icon */
#hamburger {
    cursor: pointer;
    font-size: 22px;
    margin-right: 10px;
}

/* Menu Panel */
#menuPanel {
    display: none;
    position: absolute;
    width: 180px;
    background: #111;
    top: 70px;
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 0 10px black;
    z-index: 999;
}
#menuPanel div {
    padding: 10px;
    cursor: pointer;
}
#menuPanel div:hover {
    background: #333;
}

/* Modal Windows */
.modal {
    display: none;
    position: fixed;
    top: 50%; left: 50%;
    transform: translate(-50%, -50%);
    background: #111;
    padding: 20px;
    width: 300px;
    border-radius: 10px;
    box-shadow: 0 0 14px black;
    z-index: 9999;
}
.modal button {
    margin-top: 20px;
    padding: 10px 18px;
    border: none;
    border-radius: 6px;
    cursor: pointer;
}

/* Main Converter Box */
#box {
    padding: 20px;
    text-align: center;
}

input, select {
    width: 80%;
    padding: 10px;
    margin: 8px;
    font-size: 16px;
    border-radius: 6px;
    border: none;
}

/* Buttons */
button {
    padding: 12px 20px;
    font-size: 16px;
    cursor: pointer;
    background: #00aaff;
    border: none;
    border-radius: 6px;
}
button:hover { background: #22bbff; }

</style>
</head>

<body>

<!-- Hamburger Menu Panel -->
<div id="menuPanel">
    <div onclick="openHelp()">Help</div>
    <div onclick="openAbout()">About</div>
</div>

<!-- Help Modal -->
<div id="helpModal" class="modal">
    <h3>Help</h3>
    <p>Enter an amount, choose currencies, then press Convert.<br><br>
       Rates are fixed for consistent business calculations.</p>
    <button onclick="closeHelp()">Close</button>
</div>

<!-- About Modal -->
<div id="aboutModal" class="modal">
    <h3>About This App</h3>
    <p>This is a simple business currency converter.<br>
       Built using HTML, CSS, and JavaScript.</p>
    <button onclick="closeAbout()">Close</button>
</div>

<!-- Main Application Window -->
<div id="window">

    <div id="titleBar">
        <div style="display:flex;align-items:center;">
            <div id="hamburger">☰</div>
            <span>Currency Converter</span>
        </div>
        <div id="closeBtn" onclick="closeWindow()">✕</div>
    </div>

    <div id="box">
        <h2>Convert Currency</h2>

        <input id="amount" type="number" placeholder="Enter amount">

        <select id="fromCurr">
            <option>USD</option>
            <option>EUR</option>
            <option>GBP</option>
            <option>INR</option>
        </select>

        <select id="toCurr">
            <option>USD</option>
            <option>EUR</option>
            <option>GBP</option>
            <option>INR</option>
        </select>

        <button onclick="convert()">Convert</button>

        <h3 id="result"></h3>
    </div>
</div>

<script>
/* Fixed conversion rates relative to USD */
const rates = {
    USD: 1.00,
    EUR: 0.92,
    GBP: 0.79,
    INR: 83.10
};

/* Conversion Function */
function convert() {
    let amount = parseFloat(document.getElementById("amount").value);
    let f = document.getElementById("fromCurr").value;
    let t = document.getElementById("toCurr").value;

    if (isNaN(amount)) {
        document.getElementById("result").innerHTML = "Please enter a valid amount.";
        return;
    }

    let usd = amount / rates[f];
    let converted = usd * rates[t];

    document.getElementById("result").innerHTML =
        amount + " " + f + " = " + converted.toFixed(2) + " " + t;
}

/* Close Window */
function closeWindow() {
    document.getElementById("window").style.display = "none";
}

/* Hamburger Menu Toggle */
document.getElementById("hamburger").onclick = () => {
    let menu = document.getElementById("menuPanel");
    menu.style.display = (menu.style.display === "block") ? "none" : "block";
};

/* Modal Controls */
function openHelp() {
    document.getElementById("helpModal").style.display = "block";
    document.getElementById("menuPanel").style.display = "none";
}
function closeHelp() {
    document.getElementById("helpModal").style.display = "none";
}
function openAbout() {
    document.getElementById("aboutModal").style.display = "block";
    document.getElementById("menuPanel").style.display = "none";
}
function closeAbout() {
    document.getElementById("aboutModal").style.display = "none";
}
</script>

</body>
</html>
""")


In [14]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
<title>Currency Converter App</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: #222;
    color: white;
    display: flex;
    justify-content: center;
    padding-top: 40px;
}

/* Main window */
#window {
    width: 450px;
    background: #333;
    border-radius: 10px;
    box-shadow: 0 0 12px black;
    overflow: hidden;
    position: relative;
}

/* Title Bar */
#titleBar {
    background: #444;
    padding: 10px;
    display: flex;
    align-items: center;
    justify-content: space-between;
}
#titleBar span { font-size: 17px; font-weight: bold; }

#closeBtn {
    cursor: pointer;
    font-size: 20px;
    padding: 4px 12px;
    border-radius: 5px;
    background: #666;
}
#closeBtn:hover { background: #888; }

/* Hamburger Icon */
#hamburger {
    cursor: pointer;
    font-size: 22px;
    margin-right: 10px;
}

/* Menu Panel */
#menuPanel {
    display: none;
    position: absolute;
    width: 180px;
    background: #111;
    top: 55px;
    left: 10px;
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 0 10px black;
    z-index: 999;
}
#menuPanel div {
    padding: 10px;
    cursor: pointer;
}
#menuPanel div:hover { background: #333; }

/* Modal Windows */
.modal {
    display: none;
    position: fixed;
    top: 50%; left: 50%;
    transform: translate(-50%, -50%);
    background: #111;
    padding: 20px;
    width: 320px;
    border-radius: 10px;
    box-shadow: 0 0 14px black;
    z-index: 9999;
}
.modal button {
    margin-top: 15px;
    padding: 10px 18px;
    border: none;
    border-radius: 6px;
    cursor: pointer;
}

/* Main Content Box */
#box {
    padding: 20px;
    text-align: center;
}

input, select {
    width: 80%;
    padding: 10px;
    margin: 8px;
    font-size: 16px;
    border-radius: 6px;
    border: none;
}

.flag {
    font-size: 24px;
    margin-right: 8px;
}

/* Buttons */
button {
    padding: 12px 20px;
    font-size: 16px;
    cursor: pointer;
    background: #00aaff;
    border: none;
    border-radius: 6px;
}
button:hover { background: #22bbff; }
</style>
</head>

<body>

<!-- Hamburger Menu Panel -->
<div id="menuPanel">
    <div onclick="openHelp()">Help</div>
    <div onclick="openAbout()">About</div>
</div>

<!-- Help Modal -->
<div id="helpModal" class="modal">
    <h3>Help</h3>
    <p>Enter an amount, choose currencies, and press Convert.<br><br>
       This version fetches live exchange rates from exchangerate.host.</p>
    <button onclick="closeHelp()">Close</button>
</div>

<!-- About Modal -->
<div id="aboutModal" class="modal">
    <h3>About This App</h3>
    <p>A real-time business currency converter with flag icons, a custom UI,
       and hamburger menu.</p>
    <button onclick="closeAbout()">Close</button>
</div>

<!-- Main Application Window -->
<div id="window">

    <div id="titleBar">
        <div style="display:flex;align-items:center;">
            <div id="hamburger">☰</div>
            <span>Currency Converter</span>
        </div>
        <div id="closeBtn" onclick="closeWindow()">✕</div>
    </div>

    <div id="box">
        <h2>Convert Currency</h2>

        <input id="amount" type="number" placeholder="Enter amount">

        <!-- FROM CURRENCY -->
        <div style="display:flex;align-items:center;justify-content:center;">
            <span class='flag' id="flagFrom">🇺🇸</span>
            <select id="fromCurr" onchange="updateFlagFrom()">
                <option value="USD">USD - United States</option>
                <option value="EUR">EUR - European Union</option>
                <option value="GBP">GBP - United Kingdom</option>
                <option value="INR">INR - India</option>
            </select>
        </div>

        <!-- TO CURRENCY -->
        <div style="display:flex;align-items:center;justify-content:center;">
            <span class='flag' id="flagTo">🇪🇺</span>
            <select id="toCurr" onchange="updateFlagTo()">
                <option value="USD">USD - United States</option>
                <option value="EUR" selected>EUR - European Union</option>
                <option value="GBP">GBP - United Kingdom</option>
                <option value="INR">INR - India</option>
            </select>
        </div>

        <button onclick="convert()">Convert (Live Rates)</button>

        <h3 id="result"></h3>
    </div>
</div>

<script>
/* Currency Flags */
const flags = {
    USD: "🇺🇸",
    EUR: "🇪🇺",
    GBP: "🇬🇧",
    INR: "🇮🇳"
};

function updateFlagFrom() {
    let curr = document.getElementById("fromCurr").value;
    document.getElementById("flagFrom").innerHTML = flags[curr];
}
function updateFlagTo() {
    let curr = document.getElementById("toCurr").value;
    document.getElementById("flagTo").innerHTML = flags[curr];
}

/* Real-Time Conversion Function */
async function convert() {
    let amount = parseFloat(document.getElementById("amount").value);
    let f = document.getElementById("fromCurr").value;
    let t = document.getElementById("toCurr").value;

    if (isNaN(amount)) {
        document.getElementById("result").innerHTML = "Enter a valid amount.";
        return;
    }

    let url = `https://api.exchangerate.host/convert?from=${f}&to=${t}`;
    let response = await fetch(url);
    let data = await response.json();

    let rate = data.result;
    let converted = amount * rate;

    document.getElementById("result").innerHTML =
        `${flags[f]} ${amount} ${f} = ${flags[t]} ${converted.toFixed(2)} ${t}<br><br>` +
        `<small>Live Rate: 1 ${f} = ${rate.toFixed(4)} ${t}</small>`;
}

/* Window Close */
function closeWindow() {
    document.getElementById("window").style.display = "none";
}

/* Hamburger Menu */
document.getElementById("hamburger").onclick = () => {
    let menu = document.getElementById("menuPanel");
    menu.style.display = (menu.style.display === "block") ? "none" : "block";
};

/* Modal Functions */
function openHelp() {
    document.getElementById("helpModal").style.display = "block";
    document.getElementById("menuPanel").style.display = "none";
}
function closeHelp() { document.getElementById("helpModal").style.display = "none"; }

function openAbout() {
    document.getElementById("aboutModal").style.display = "block";
    document.getElementById("menuPanel").style.display = "none";
}
function closeAbout() { document.getElementById("aboutModal").style.display = "none"; }

</script>

</body>
</html>
""")


In [17]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
<title>Currency Converter App</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: url('https://images.unsplash.com/photo-1526304640581-d334cdbbf45e?w=1200')
        no-repeat center center fixed;
    background-size: cover;
    color: white;
    display: flex;
    justify-content: center;
    padding-top: 40px;
}

/* MAIN WINDOW */
#window {
    width: 480px;
    background: rgba(0,0,0,0.75);
    border-radius: 12px;
    box-shadow: 0 0 18px black;
    overflow: hidden;
    backdrop-filter: blur(5px);
    position: absolute;
    top: 140px;
    left: 200px;
}

/* TITLE BAR (DRAGGABLE) */
#titleBar {
    background: rgba(40,40,40,0.85);
    padding: 10px;
    display: flex;
    justify-content: space-between;
    align-items: center;
    cursor: move;
}

#hamburger {
    font-size: 22px;
    cursor: pointer;
}

#closeBtn {
    cursor: pointer;
    font-size: 20px;
    padding: 4px 12px;
    border-radius: 5px;
    background: #666;
}
#closeBtn:hover { background: #888; }

/* ANIMATED SLIDE MENU */
#menuPanel {
    position: absolute;
    width: 200px;
    background: rgba(0,0,0,0.88);
    left: -240px;
    top: 60px;
    padding: 12px;
    border-radius: 10px;
    transition: left 0.35s ease;
    box-shadow: 0 0 14px black;
    z-index: 999;
}

#menuPanel.show {
    left: 15px;
}

#menuPanel div {
    padding: 12px;
    cursor: pointer;
}
#menuPanel div:hover { background: #444; }

/* MODALS */
.modal {
    display: none;
    position: fixed;
    top: 50%; left: 50%;
    transform: translate(-50%, -50%);
    background: rgba(0,0,0,0.92);
    padding: 20px;
    width: 330px;
    border-radius: 10px;
    box-shadow: 0 0 20px black;
    z-index: 9999;
}

.modal button {
    margin-top: 15px;
    padding: 10px 18px;
    border-radius: 6px;
    background: #00aaff;
    border: none;
    cursor: pointer;
}

/* MAIN CONTENT */
#box {
    padding: 20px;
    text-align: center;
}

.inputRow {
    display: flex;
    justify-content: center;
    align-items: center;
    margin-top: 12px;
}

input.amountField {
    width: 50%;
    padding: 12px;
    font-size: 18px;
    border-radius: 6px;
    border: none;
    margin: 8px;
}

select {
    padding: 10px;
    font-size: 16px;
    border-radius: 6px;
    margin-left: 10px;
}

.flag {
    font-size: 28px;
    margin-right: 10px;
}

/* VERTICAL SWAP BUTTON */
#swapBtn {
    font-size: 35px;
    cursor: pointer;
    margin: 0 10px;
    user-select: none;
    padding: 10px;
}
#swapBtn:hover {
    color: #00aaff;
    transform: scale(1.2);
}

/* CLEAN RATE DISPLAY */
#result {
    margin-top: 20px;
    font-size: 18px;
}
</style>
</head>

<body>

<!-- SLIDE MENU -->
<div id="menuPanel">
    <div onclick="openHelp()">Help</div>
    <div onclick="openAbout()">About</div>
</div>

<!-- HELP MODAL -->
<div id="helpModal" class="modal">
    <h3>Help</h3>
    <p>
        Enter values in either FROM or TO fields.<br>
        Editing FROM updates TO.<br>
        Editing TO updates FROM.<br><br>
        Use ⇅ to swap currencies.
    </p>
    <button onclick="closeHelp()">Close</button>
</div>

<!-- ABOUT MODAL -->
<div id="aboutModal" class="modal">
    <h3>About</h3>
    <p>
        Live two-way currency converter with draggable UI, sliding menu, flags,
        and real-time API integration.
    </p>
    <button onclick="closeAbout()">Close</button>
</div>

<!-- MAIN WINDOW -->
<div id="window">

    <div id="titleBar">
        <div style="display:flex;align-items:center;">
            <span id="hamburger">☰</span>
            <span style="margin-left:10px;font-weight:bold;">Currency Converter</span>
        </div>
        <div id="closeBtn" onclick="closeWindow()">✕</div>
    </div>

    <div id="box">

        <h2>Live Currency Converter</h2>

        <!-- FROM FIELD -->
        <div class="inputRow">
            <span class="flag" id="flagFrom">🇺🇸</span>
            <input class="amountField" id="amountFrom" type="number" placeholder="Amount"
                   oninput="convertFrom()">
            <select id="fromCurr" onchange="updateFlagFrom(); convertFrom();">
                <option value="USD">USD</option>
                <option value="EUR">EUR</option>
                <option value="GBP">GBP</option>
                <option value="INR">INR</option>
            </select>
        </div>

        <!-- SWAP BUTTON -->
        <div style="text-align:center;margin:5px;">
            <span id="swapBtn" onclick="swapCurrencies()">⇅</span>
        </div>

        <!-- TO FIELD -->
        <div class="inputRow">
            <span class="flag" id="flagTo">🇪🇺</span>
            <input class="amountField" id="amountTo" type="number" placeholder="Amount"
                   oninput="convertTo()">
            <select id="toCurr" onchange="updateFlagTo(); convertFrom();">
                <option value="USD">USD</option>
                <option value="EUR" selected>EUR</option>
                <option value="GBP">GBP</option>
                <option value="INR">INR</option>
            </select>
        </div>

        <div id="result"></div>

    </div>
</div>

<script>
/* FLAGS */
const flags = { USD:"🇺🇸", EUR:"🇪🇺", GBP:"🇬🇧", INR:"🇮🇳" };

function updateFlagFrom(){ flagFrom.textContent = flags[fromCurr.value]; }
function updateFlagTo(){ flagTo.textContent = flags[toCurr.value]; }

/* LIVE RATES API */
async function getRate(base) {
    let url = `https://open.er-api.com/v6/latest/${base}`;
    let response = await fetch(url);
    let data = await response.json();
    return data.rates;
}

/* FROM → TO */
async function convertFrom() {
    let amount = parseFloat(amountFrom.value);
    if (isNaN(amount)) return;

    let from = fromCurr.value;
    let to = toCurr.value;

    let rates = await getRate(from);
    amountTo.value = (amount * rates[to]).toFixed(4);

    showRate(from, to, rates[to]);
}

/* TO → FROM */
async function convertTo() {
    let amount = parseFloat(amountTo.value);
    if (isNaN(amount)) return;

    let from = fromCurr.value;
    let to = toCurr.value;

    let rates = await getRate(to);
    amountFrom.value = (amount * rates[from]).toFixed(4);

    showRate(from, to, 1 / rates[from]);
}

/* CLEAN OUTPUT — ONLY RATE */
function showRate(from, to, rate) {
    result.innerHTML = `<small>1 ${from} = ${rate.toFixed(4)} ${to}</small>`;
}

/* SWAP CURRENCIES + VALUES */
function swapCurrencies() {
    let tempC = fromCurr.value;
    fromCurr.value = toCurr.value;
    toCurr.value = tempC;

    updateFlagFrom();
    updateFlagTo();

    let tempA = amountFrom.value;
    amountFrom.value = amountTo.value;
    amountTo.value = tempA;

    convertFrom();
}

/* CLOSE WINDOW */
function closeWindow(){
    window.style.display = "none";
}

/* SLIDE MENU */
hamburger.onclick = () => menuPanel.classList.toggle("show");

/* MODALS */
function openHelp(){ helpModal.style.display="block"; }
function closeHelp(){ helpModal.style.display="none"; }
function openAbout(){ aboutModal.style.display="block"; }
function closeAbout(){ aboutModal.style.display="none"; }

/* DRAGGABLE WINDOW */
let dragging = false, offX = 0, offY = 0;

titleBar.onmousedown = (e) => {
    dragging = true;
    offX = e.clientX - window.offsetLeft;
    offY = e.clientY - window.offsetTop;
};

document.onmouseup = () => dragging = false;

document.onmousemove = (e) => {
    if (dragging) {
        window.style.left = (e.clientX - offX) + "px";
        window.style.top  = (e.clientY - offY) + "px";
    }
};
</script>

</body>
</html>
""")


In [20]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
<title>Currency Converter App</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: url('https://images.unsplash.com/photo-1567427017947-545c5f8d16ad?w=1600')
        no-repeat center center fixed;
    background-size: cover;
    color: white;
    display: flex;
    justify-content: center;
    padding-top: 40px;
    overflow: hidden;
}

/* MAIN WINDOW */
#appWindow {
    width: 480px;
    background: rgba(0,0,0,0.75);
    border-radius: 12px;
    box-shadow: 0 0 18px black;
    overflow: hidden;
    position: absolute;
    top: 140px;
    left: 200px;
    backdrop-filter: blur(6px);
    z-index: 10;
}

/* TITLE BAR (DRAGGABLE) */
#titleBar {
    background: rgba(40,40,40,0.85);
    padding: 10px;
    display: flex;
    justify-content: space-between;
    align-items: center;
    cursor: move;
}

#hamburger {
    font-size: 22px;
    cursor: pointer;
}

#closeBtn {
    cursor: pointer;
    font-size: 20px;
    padding: 4px 12px;
    border-radius: 5px;
    background: #666;
}
#closeBtn:hover { background: #888; }

/* SLIDE MENU */
#menuPanel {
    position: absolute;
    width: 200px;
    background: rgba(0,0,0,0.88);
    left: -240px;
    top: 60px;
    padding: 12px;
    border-radius: 10px;
    transition: left 0.35s ease;
    box-shadow: 0 0 14px black;
    z-index: 999;
}

#menuPanel.show {
    left: 15px;
}

#menuPanel div {
    padding: 12px;
    cursor: pointer;
}
#menuPanel div:hover { background: #444; }

/* MODALS (INSIDE APP WINDOW) */
.modal {
    display: none;
    position: absolute;
    top: 50%; left: 50%;
    transform: translate(-50%, -50%);
    background: rgba(0,0,0,0.92);
    padding: 20px;
    width: 330px;
    border-radius: 10px;
    box-shadow: 0 0 20px black;
    z-index: 2000;
}

.modal button {
    margin-top: 15px;
    padding: 10px 18px;
    border-radius: 6px;
    background: #00aaff;
    border: none;
    cursor: pointer;
}

/* MAIN CONTENT */
#box {
    padding: 20px;
    text-align: center;
}

.inputRow {
    display: flex;
    justify-content: center;
    align-items: center;
    margin-top: 12px;
}

input.amountField {
    width: 50%;
    padding: 12px;
    font-size: 18px;
    border-radius: 6px;
    border: none;
    margin: 8px;
}

select {
    padding: 10px;
    font-size: 16px;
    border-radius: 6px;
    margin-left: 10px;
}

.flag {
    font-size: 28px;
    margin-right: 10px;
}

/* VERTICAL SWAP BUTTON */
#swapBtn {
    font-size: 35px;
    cursor: pointer;
    margin: 0 10px;
    user-select: none;
}
#swapBtn:hover {
    color: #00aaff;
    transform: scale(1.2);
}

/* CLEAN RATE DISPLAY */
#result {
    margin-top: 20px;
    font-size: 18px;
}
</style>
</head>

<body>

<!-- SLIDE MENU -->
<div id="menuPanel">
    <div onclick="openHelp()">Help</div>
    <div onclick="openAbout()">About</div>
</div>

<!-- HELP MODAL -->
<div id="helpModal" class="modal">
    <h3>Help</h3>
    <p>
        You may type in either field.<br>
        FROM updates TO.<br>
        TO updates FROM.<br><br>
        ⇅ swaps currencies instantly.
    </p>
    <button onclick="closeHelp()">Close</button>
</div>

<!-- ABOUT MODAL -->
<div id="aboutModal" class="modal">
    <h3>About</h3>
    <p>
        Live two-way currency converter with draggable UI,<br>
        animated menu, euro background and real-time API.
    </p>
    <button onclick="closeAbout()">Close</button>
</div>

<!-- MAIN APP WINDOW -->
<div id="appWindow">

    <div id="titleBar">
        <div style="display:flex;align-items:center;">
            <span id="hamburger">☰</span>
            <span style="margin-left:10px;font-weight:bold;">Currency Converter</span>
        </div>
        <div id="closeBtn" onclick="closeWindow()">✕</div>
    </div>

    <div id="box">

        <h2>Live Currency Converter</h2>

        <!-- FROM FIELD -->
        <div class="inputRow">
            <span class="flag" id="flagFrom">🇺🇸</span>
            <input class="amountField" id="amountFrom" type="number" placeholder="Amount"
                   oninput="convertFrom()">
            <select id="fromCurr" onchange="updateFlagFrom(); convertFrom();">
                <option value="USD">USD</option>
                <option value="EUR">EUR</option>
                <option value="GBP">GBP</option>
                <option value="INR">INR</option>
            </select>
        </div>

        <!-- SWAP BUTTON -->
        <div style="text-align:center;margin:5px;">
            <span id="swapBtn" onclick="swapCurrencies()">⇅</span>
        </div>

        <!-- TO FIELD -->
        <div class="inputRow">
            <span class="flag" id="flagTo">🇪🇺</span>
            <input class="amountField" id="amountTo" type="number" placeholder="Amount"
                   oninput="convertTo()">
            <select id="toCurr" onchange="updateFlagTo(); convertFrom();">
                <option value="USD">USD</option>
                <option value="EUR" selected>EUR</option>
                <option value="GBP">GBP</option>
                <option value="INR">INR</option>
            </select>
        </div>

        <div id="result"></div>

    </div>
</div>

<script>
/* FLAGS */
const flags = { USD:"🇺🇸", EUR:"🇪🇺", GBP:"🇬🇧", INR:"🇮🇳" };

function updateFlagFrom(){ flagFrom.textContent = flags[fromCurr.value]; }
function updateFlagTo(){ flagTo.textContent = flags[toCurr.value]; }

/* LIVE RATES API */
async function getRate(base) {
    let url = `https://open.er-api.com/v6/latest/${base}`;
    let response = await fetch(url);
    let data = await response.json();
    return data.rates;
}

/* FROM → TO */
async function convertFrom() {
    let amount = parseFloat(amountFrom.value);
    if (isNaN(amount)) return;

    let rates = await getRate(fromCurr.value);
    let rate = rates[toCurr.value];

    amountTo.value = (amount * rate).toFixed(4);
    result.innerHTML = `<small>1 ${fromCurr.value} = ${rate.toFixed(4)} ${toCurr.value}</small>`;
}

/* TO → FROM */
async function convertTo() {
    let amount = parseFloat(amountTo.value);
    if (isNaN(amount)) return;

    let rates = await getRate(toCurr.value);
    let rate = rates[fromCurr.value];

    amountFrom.value = (amount * rate).toFixed(4);
    result.innerHTML = `<small>1 ${toCurr.value} = ${rate.toFixed(4)} ${fromCurr.value}</small>`;
}

/* SWAP CURRENCIES + VALUES */
function swapCurrencies() {
    let tempC = fromCurr.value;
    fromCurr.value = toCurr.value;
    toCurr.value = tempC;

    updateFlagFrom();
    updateFlagTo();

    let tempA = amountFrom.value;
    amountFrom.value = amountTo.value;
    amountTo.value = tempA;

    convertFrom();
}

/* FIXED CLOSE FUNCTION */
function closeWindow(){
    document.getElementById("appWindow").style.display = "none";
}

/* SLIDE MENU */
hamburger.onclick = () => menuPanel.classList.toggle("show");

/* MODALS */
function openHelp(){ helpModal.style.display="block"; }
function closeHelp(){ helpModal.style.display="none"; }
function openAbout(){ aboutModal.style.display="block"; }
function closeAbout(){ aboutModal.style.display="none"; }

/* DRAGGABLE WINDOW */
let dragging = false, offX = 0, offY = 0;
let app = document.getElementById("appWindow");

titleBar.onmousedown = (e) => {
    dragging = true;
    offX = e.clientX - app.offsetLeft;
    offY = e.clientY - app.offsetTop;
};

document.onmouseup = () => dragging = false;

document.onmousemove = (e) => {
    if (dragging) {
        app.style.left = (e.clientX - offX) + "px";
        app.style.top  = (e.clientY - offY) + "px";
    }
};
</script>

</body>
</html>
""")


In [21]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
<title>Currency Converter App</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: url('https://images.unsplash.com/photo-1567427017947-545c5f8d16ad?w=1600')
        no-repeat center center fixed;
    background-size: cover;
    color: white;
    display: flex;
    justify-content: center;
    padding-top: 40px;
    overflow: hidden;
}

/* MAIN WINDOW */
#appWindow {
    width: 480px;
    background: rgba(0,0,0,0.75);
    border-radius: 12px;
    box-shadow: 0 0 18px black;
    overflow: hidden;
    position: absolute;
    top: 140px;
    left: 200px;
    backdrop-filter: blur(6px);
    z-index: 10;
}

/* TITLE BAR (DRAGGABLE) */
#titleBar {
    background: rgba(40,40,40,0.85);
    padding: 10px;
    display: flex;
    justify-content: space-between;
    align-items: center;
    cursor: move;
}

#hamburger {
    font-size: 22px;
    cursor: pointer;
}

#closeBtn {
    cursor: pointer;
    font-size: 20px;
    padding: 4px 12px;
    border-radius: 5px;
    background: #666;
}
#closeBtn:hover { background: #888; }

/* SLIDE MENU */
#menuPanel {
    position: absolute;
    width: 200px;
    background: rgba(0,0,0,0.88);
    left: -240px;
    top: 60px;
    padding: 12px;
    border-radius: 10px;
    transition: left 0.35s ease;
    box-shadow: 0 0 14px black;
    z-index: 999;
}

#menuPanel.show { left: 15px; }

#menuPanel div {
    padding: 12px;
    cursor: pointer;
}
#menuPanel div:hover { background: #444; }

/* MODALS centered INSIDE window */
.modal {
    display: none;
    position: absolute;
    top: 50%; left: 50%;
    transform: translate(-50%, -50%);
    background: rgba(0,0,0,0.92);
    padding: 20px;
    width: 330px;
    border-radius: 10px;
    box-shadow: 0 0 20px black;
    z-index: 2000;
}

.modal button {
    margin-top: 15px;
    padding: 10px 18px;
    border-radius: 6px;
    background: #00aaff;
    border: none;
    cursor: pointer;
}

/* MAIN CONTENT */
#box {
    padding: 20px;
    text-align: center;
}

.inputRow {
    display: flex;
    justify-content: center;
    align-items: center;
    margin-top: 12px;
}

input.amountField {
    width: 50%;
    padding: 12px;
    font-size: 18px;
    border-radius: 6px;
    border: none;
    margin: 8px;
}

select {
    padding: 10px;
    font-size: 16px;
    border-radius: 6px;
    margin-left: 10px;
}

.flag {
    font-size: 28px;
    margin-right: 10px;
}

/* VERTICAL SWAP BUTTON */
#swapBtn {
    font-size: 35px;
    cursor: pointer;
    margin: 0 10px;
    user-select: none;
}
#swapBtn:hover {
    color: #00aaff;
    transform: scale(1.2);
}

/* RATE DISPLAY */
#result {
    margin-top: 20px;
    font-size: 18px;
}
</style>
</head>

<body>

<!-- SLIDE MENU -->
<div id="menuPanel">
    <div onclick="openHelp()">Help</div>
    <div onclick="openAbout()">About</div>
</div>

<!-- HELP MODAL -->
<div id="helpModal" class="modal">
    <h3>Help</h3>
    <p>
        Type in either box.<br>
        FROM updates TO.<br>
        TO updates FROM.<br><br>
        ⇅ swaps currencies instantly.
    </p>
    <button onclick="closeHelp()">Close</button>
</div>

<!-- ABOUT MODAL -->
<div id="aboutModal" class="modal">
    <h3>About</h3>
    <p>
        Live two-way currency converter with flags, draggable window,<br>
        animated menu, and euro-themed background.
    </p>
    <button onclick="closeAbout()">Close</button>
</div>

<!-- MAIN APP WINDOW -->
<div id="appWindow">

    <div id="titleBar">
        <div style="display:flex;align-items:center;">
            <span id="hamburger">☰</span>
            <span style="margin-left:10px;font-weight:bold;">Currency Converter</span>
        </div>
        <div id="closeBtn" onclick="closeWindow()">✕</div>
    </div>

    <div id="box">

        <h2>Live Currency Converter</h2>

        <!-- FROM FIELD -->
        <div class="inputRow">
            <span class="flag" id="flagFrom">🇺🇸</span>

            <input class="amountField" id="amountFrom" type="number"
                   placeholder="Amount" oninput="convertFrom()">

            <select id="fromCurr" onchange="updateFlagFrom(); convertFrom();">
                <option value="USD">🇺🇸 USD — United States</option>
                <option value="EUR">🇪🇺 EUR — European Union</option>
                <option value="GBP">🇬🇧 GBP — United Kingdom</option>
                <option value="INR">🇮🇳 INR — India</option>
            </select>
        </div>

        <!-- SWAP BUTTON -->
        <div style="text-align:center;margin:5px;">
            <span id="swapBtn" onclick="swapCurrencies()">⇅</span>
        </div>

        <!-- TO FIELD -->
        <div class="inputRow">
            <span class="flag" id="flagTo">🇪🇺</span>

            <input class="amountField" id="amountTo" type="number"
                   placeholder="Amount" oninput="convertTo()">

            <select id="toCurr" onchange="updateFlagTo(); convertFrom();">
                <option value="USD">🇺🇸 USD — United States</option>
                <option value="EUR" selected>🇪🇺 EUR — European Union</option>
                <option value="GBP">🇬🇧 GBP — United Kingdom</option>
                <option value="INR">🇮🇳 INR — India</option>
            </select>
        </div>

        <div id="result"></div>

    </div>
</div>

<script>
/* FLAG MAP */
const flags = {
    USD:"🇺🇸", EUR:"🇪🇺", GBP:"🇬🇧", INR:"🇮🇳"
};

function updateFlagFrom(){ flagFrom.textContent = flags[fromCurr.value]; }
function updateFlagTo(){ flagTo.textContent = flags[toCurr.value]; }

/* LIVE API */
async function getRate(base) {
    let url = `https://open.er-api.com/v6/latest/${base}`;
    let response = await fetch(url);
    let data = await response.json();
    return data.rates;
}

/* FROM → TO conversion */
async function convertFrom() {
    let amount = parseFloat(amountFrom.value);
    if (isNaN(amount)) return;

    let rates = await getRate(fromCurr.value);
    let rate = rates[toCurr.value];

    amountTo.value = (amount * rate).toFixed(4);
    result.innerHTML = `<small>1 ${fromCurr.value} = ${rate.toFixed(4)} ${toCurr.value}</small>`;
}

/* TO → FROM conversion */
async function convertTo() {
    let amount = parseFloat(amountTo.value);
    if (isNaN(amount)) return;

    let rates = await getRate(toCurr.value);
    let rate = rates[fromCurr.value];

    amountFrom.value = (amount * rate).toFixed(4);
    result.innerHTML = `<small>1 ${toCurr.value} = ${rate.toFixed(4)} ${fromCurr.value}</small>`;
}

/* SWAP BUTTON */
function swapCurrencies() {
    let c = fromCurr.value;
    fromCurr.value = toCurr.value;
    toCurr.value = c;

    updateFlagFrom();
    updateFlagTo();

    let a = amountFrom.value;
    amountFrom.value = amountTo.value;
    amountTo.value = a;

    convertFrom();
}

/* CLOSE WINDOW */
function closeWindow(){
    appWindow.style.display = "none";
}

/* SLIDE MENU */
hamburger.onclick = () => { menuPanel.classList.toggle("show"); };

/* MODALS */
function openHelp(){ helpModal.style.display="block"; }
function closeHelp(){ helpModal.style.display="none"; }
function openAbout(){ aboutModal.style.display="block"; }
function closeAbout(){ aboutModal.style.display="none"; }

/* DRAGGABLE WINDOW */
let dragging = false, offX = 0, offY = 0;

titleBar.onmousedown = (e) => {
    dragging = true;
    offX = e.clientX - appWindow.offsetLeft;
    offY = e.clientY - appWindow.offsetTop;
};

document.onmouseup = () => dragging = false;

document.onmousemove = (e) => {
    if (!dragging) return;
    appWindow.style.left = (e.clientX - offX) + "px";
    appWindow.style.top = (e.clientY - offY) + "px";
};
</script>

</body>
</html>
""")


In [ ]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
<title>Currency Converter App</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: url('https://images.unsplash.com/photo-1567427017947-545c5f8d16ad?w=1600')
        no-repeat center center fixed;
    background-size: cover;
    color: white;
    display: flex;
    justify-content: center;
    padding-top: 40px;
    overflow: hidden;
}

/* MAIN WINDOW */
#appWindow {
    width: 480px;
    background: rgba(0,0,0,0.75);
    border-radius: 12px;
    box-shadow: 0 0 18px black;
    overflow: hidden;
    position: absolute;
    top: 140px;
    left: 200px;
    backdrop-filter: blur(6px);
    z-index: 10;
}

/* TITLE BAR (DRAGGABLE) */
#titleBar {
    background: rgba(40,40,40,0.85);
    padding: 10px;
    display: flex;
    justify-content: space-between;
    align-items: center;
    cursor: move;
}

#hamburger {
    font-size: 22px;
    cursor: pointer;
}

#closeBtn {
    cursor: pointer;
    font-size: 20px;
    padding: 4px 12px;
    border-radius: 5px;
    background: #666;
}
#closeBtn:hover { background: #888; }

/* SLIDE MENU */
#menuPanel {
    position: absolute;
    width: 200px;
    background: rgba(0,0,0,0.88);
    left: -240px;
    top: 60px;
    padding: 12px;
    border-radius: 10px;
    transition: left 0.35s ease;
    box-shadow: 0 0 14px black;
    z-index: 999;
}

#menuPanel.show {
    left: 15px;
}

#menuPanel div {
    padding: 12px;
    cursor: pointer;
}
#menuPanel div:hover { background: #444; }

/* MODALS (INSIDE APP WINDOW) */
.modal {
    display: none;
    position: absolute;
    top: 50%; left: 50%;
    transform: translate(-50%, -50%);
    background: rgba(0,0,0,0.92);
    padding: 20px;
    width: 330px;
    border-radius: 10px;
    box-shadow: 0 0 20px black;
    z-index: 2000;
}

.modal button {
    margin-top: 15px;
    padding: 10px 18px;
    border-radius: 6px;
    background: #00aaff;
    border: none;
    cursor: pointer;
}

/* MAIN CONTENT */
#box {
    padding: 20px;
    text-align: center;
}

.inputRow {
    display: flex;
    justify-content: center;
    align-items: center;
    margin-top: 12px;
}

input.amountField {
    width: 50%;
    padding: 12px;
    font-size: 18px;
    border-radius: 6px;
    border: none;
    margin: 8px;
}

select {
    padding: 10px;
    font-size: 16px;
    border-radius: 6px;
    margin-left: 10px;
}

.flag {
    font-size: 28px;
    margin-right: 10px;
}

/* VERTICAL SWAP BUTTON */
#swapBtn {
    font-size: 35px;
    cursor: pointer;
    margin: 0 10px;
    user-select: none;
}
#swapBtn:hover {
    color: #00aaff;
    transform: scale(1.2);
}

/* CLEAN RATE DISPLAY */
#result {
    margin-top: 20px;
    font-size: 18px;
}
</style>
</head>

<body>

<!-- SLIDE MENU -->
<div id="menuPanel">
    <div onclick="openHelp()">Help</div>
    <div onclick="openAbout()">About</div>
</div>

<!-- HELP MODAL -->
<div id="helpModal" class="modal">
    <h3>Help</h3>
    <p>
        You may type in either field.<br>
        FROM updates TO.<br>
        TO updates FROM.<br><br>
        ⇅ swaps currencies instantly.
    </p>
    <button onclick="closeHelp()">Close</button>
</div>

<!-- ABOUT MODAL -->
<div id="aboutModal" class="modal">
    <h3>About</h3>
    <p>
        Live two-way currency converter with draggable UI,<br>
        animated menu, euro background and real-time API.
    </p>
    <button onclick="closeAbout()">Close</button>
</div>

<!-- MAIN APP WINDOW -->
<div id="appWindow">

    <div id="titleBar">
        <div style="display:flex;align-items:center;">
            <span id="hamburger">☰</span>
            <span style="margin-left:10px;font-weight:bold;">Currency Converter</span>
        </div>
        <div id="closeBtn" onclick="closeWindow()">✕</div>
    </div>

    <div id="box">

        <h2>Live Currency Converter</h2>

        <!-- FROM FIELD -->
        <div class="inputRow">
            <span class="flag" id="flagFrom">🇺🇸</span>
            <input class="amountField" id="amountFrom" type="number" placeholder="Amount"
                   oninput="convertFrom()">
            <select id="fromCurr" onchange="updateFlagFrom(); convertFrom();">
                <option value="USD">USD</option>
                <option value="EUR">EUR</option>
                <option value="GBP">GBP</option>
                <option value="INR">INR</option>
            </select>
        </div>

        <!-- SWAP BUTTON -->
        <div style="text-align:center;margin:5px;">
            <span id="swapBtn" onclick="swapCurrencies()">⇅</span>
        </div>

        <!-- TO FIELD -->
        <div class="inputRow">
            <span class="flag" id="flagTo">🇪🇺</span>
            <input class="amountField" id="amountTo" type="number" placeholder="Amount"
                   oninput="convertTo()">
            <select id="toCurr" onchange="updateFlagTo(); convertFrom();">
                <option value="USD">USD</option>
                <option value="EUR" selected>EUR</option>
                <option value="GBP">GBP</option>
                <option value="INR">INR</option>
            </select>
        </div>

        <div id="result"></div>

    </div>
</div>

<script>
/* FLAGS */
const flags = { USD:"🇺🇸", EUR:"🇪🇺", GBP:"🇬🇧", INR:"🇮🇳" };

function updateFlagFrom(){ flagFrom.textContent = flags[fromCurr.value]; }
function updateFlagTo(){ flagTo.textContent = flags[toCurr.value]; }

/* LIVE RATES API */
async function getRate(base) {
    let url = `https://open.er-api.com/v6/latest/${base}`;
    let response = await fetch(url);
    let data = await response.json();
    return data.rates;
}

/* FROM → TO */
async function convertFrom() {
    let amount = parseFloat(amountFrom.value);
    if (isNaN(amount)) return;

    let rates = await getRate(fromCurr.value);
    let rate = rates[toCurr.value];

    amountTo.value = (amount * rate).toFixed(4);
    result.innerHTML = `<small>1 ${fromCurr.value} = ${rate.toFixed(4)} ${toCurr.value}</small>`;
}

/* TO → FROM */
async function convertTo() {
    let amount = parseFloat(amountTo.value);
    if (isNaN(amount)) return;

    let rates = await getRate(toCurr.value);
    let rate = rates[fromCurr.value];

    amountFrom.value = (amount * rate).toFixed(4);
    result.innerHTML = `<small>1 ${toCurr.value} = ${rate.toFixed(4)} ${fromCurr.value}</small>`;
}

/* SWAP CURRENCIES + VALUES */
function swapCurrencies() {
    let tempC = fromCurr.value;
    fromCurr.value = toCurr.value;
    toCurr.value = tempC;

    updateFlagFrom();
    updateFlagTo();

    let tempA = amountFrom.value;
    amountFrom.value = amountTo.value;
    amountTo.value = tempA;

    convertFrom();
}

/* FIXED CLOSE FUNCTION */
function closeWindow(){
    document.getElementById("appWindow").style.display = "none";
}

/* SLIDE MENU */
hamburger.onclick = () => menuPanel.classList.toggle("show");

/* MODALS */
function openHelp(){ helpModal.style.display="block"; }
function closeHelp(){ helpModal.style.display="none"; }
function openAbout(){ aboutModal.style.display="block"; }
function closeAbout(){ aboutModal.style.display="none"; }

/* DRAGGABLE WINDOW */
let dragging = false, offX = 0, offY = 0;
let app = document.getElementById("appWindow");

titleBar.onmousedown = (e) => {
    dragging = true;
    offX = e.clientX - app.offsetLeft;
    offY = e.clientY - app.offsetTop;
};

document.onmouseup = () => dragging = false;

document.onmousemove = (e) => {
    if (dragging) {
        app.style.left = (e.clientX - offX) + "px";
        app.style.top  = (e.clientY - offY) + "px";
    }
};
</script>

</body>
</html>
""")


In [22]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
<title>Currency Converter App</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: url('https://images.unsplash.com/photo-1567427017947-545c5f8d16ad?w=1600')
        no-repeat center center fixed;
    background-size: cover;
    color: white;
    display: flex;
    justify-content: center;
    padding-top: 40px;
    overflow: hidden;
}

/* MAIN WINDOW */
#appWindow {
    width: 480px;
    background: rgba(0,0,0,0.75);
    border-radius: 12px;
    box-shadow: 0 0 18px black;
    overflow: hidden;
    position: absolute;
    top: 140px;
    left: 200px;
    backdrop-filter: blur(6px);
    z-index: 10;
}

/* TITLE BAR (DRAGGABLE) */
#titleBar {
    background: rgba(40,40,40,0.85);
    padding: 10px;
    display: flex;
    justify-content: space-between;
    align-items: center;
    cursor: move;
}

#hamburger {
    font-size: 22px;
    cursor: pointer;
}

#closeBtn {
    cursor: pointer;
    font-size: 20px;
    padding: 4px 12px;
    border-radius: 5px;
    background: #666;
}
#closeBtn:hover { background: #888; }

/* SLIDE MENU */
#menuPanel {
    position: absolute;
    width: 200px;
    background: rgba(0,0,0,0.88);
    left: -240px;
    top: 60px;
    padding: 12px;
    border-radius: 10px;
    transition: left 0.35s ease;
    box-shadow: 0 0 14px black;
    z-index: 999;
}

#menuPanel.show {
    left: 15px;
}

#menuPanel div {
    padding: 12px;
    cursor: pointer;
}
#menuPanel div:hover { background: #444; }

/* MODALS (INSIDE APP WINDOW) */
.modal {
    display: none;
    position: absolute;
    top: 50%; left: 50%;
    transform: translate(-50%, -50%);
    background: rgba(0,0,0,0.92);
    padding: 20px;
    width: 330px;
    border-radius: 10px;
    box-shadow: 0 0 20px black;
    z-index: 2000;
}

.modal button {
    margin-top: 15px;
    padding: 10px 18px;
    border-radius: 6px;
    background: #00aaff;
    border: none;
    cursor: pointer;
}

/* MAIN CONTENT */
#box {
    padding: 20px;
    text-align: center;
}

.inputRow {
    display: flex;
    justify-content: center;
    align-items: center;
    margin-top: 12px;
}

input.amountField {
    width: 50%;
    padding: 12px;
    font-size: 18px;
    border-radius: 6px;
    border: none;
    margin: 8px;
}

select {
    padding: 10px;
    font-size: 16px;
    border-radius: 6px;
    margin-left: 10px;
}

.flag {
    font-size: 28px;
    margin-right: 10px;
}

/* VERTICAL SWAP BUTTON */
#swapBtn {
    font-size: 35px;
    cursor: pointer;
    margin: 0 10px;
    user-select: none;
}
#swapBtn:hover {
    color: #00aaff;
    transform: scale(1.2);
}

/* CLEAN RATE DISPLAY */
#result {
    margin-top: 20px;
    font-size: 18px;
}
</style>
</head>

<body>

<!-- SLIDE MENU -->
<div id="menuPanel">
    <div onclick="openHelp()">Help</div>
    <div onclick="openAbout()">About</div>
</div>

<!-- HELP MODAL -->
<div id="helpModal" class="modal">
    <h3>Help</h3>
    <p>
        You may type in either field.<br>
        FROM updates TO.<br>
        TO updates FROM.<br><br>
        ⇅ swaps currencies instantly.
    </p>
    <button onclick="closeHelp()">Close</button>
</div>

<!-- ABOUT MODAL -->
<div id="aboutModal" class="modal">
    <h3>About</h3>
    <p>
        Live two-way currency converter with draggable UI,<br>
        animated menu, euro background and real-time API.
    </p>
    <button onclick="closeAbout()">Close</button>
</div>

<!-- MAIN APP WINDOW -->
<div id="appWindow">

    <div id="titleBar">
        <div style="display:flex;align-items:center;">
            <span id="hamburger">☰</span>
            <span style="margin-left:10px;font-weight:bold;">Currency Converter</span>
        </div>
        <div id="closeBtn" onclick="closeWindow()">✕</div>
    </div>

    <div id="box">

        <h2>Live Currency Converter</h2>

        <!-- FROM FIELD -->
        <div class="inputRow">
            <span class="flag" id="flagFrom">🇺🇸</span>
            <input class="amountField" id="amountFrom" type="number" placeholder="Amount"
                   oninput="convertFrom()">
            <select id="fromCurr" onchange="updateFlagFrom(); convertFrom();">
                <option value="USD">USD</option>
                <option value="EUR">EUR</option>
                <option value="GBP">GBP</option>
                <option value="INR">INR</option>
            </select>
        </div>

        <!-- SWAP BUTTON -->
        <div style="text-align:center;margin:5px;">
            <span id="swapBtn" onclick="swapCurrencies()">⇅</span>
        </div>

        <!-- TO FIELD -->
        <div class="inputRow">
            <span class="flag" id="flagTo">🇪🇺</span>
            <input class="amountField" id="amountTo" type="number" placeholder="Amount"
                   oninput="convertTo()">
            <select id="toCurr" onchange="updateFlagTo(); convertFrom();">
                <option value="USD">USD</option>
                <option value="EUR" selected>EUR</option>
                <option value="GBP">GBP</option>
                <option value="INR">INR</option>
            </select>
        </div>

        <div id="result"></div>

    </div>
</div>

<script>
/* FLAGS */
const flags = { USD:"🇺🇸", EUR:"🇪🇺", GBP:"🇬🇧", INR:"🇮🇳" };

function updateFlagFrom(){ flagFrom.textContent = flags[fromCurr.value]; }
function updateFlagTo(){ flagTo.textContent = flags[toCurr.value]; }

/* LIVE RATES API */
async function getRate(base) {
    let url = `https://open.er-api.com/v6/latest/${base}`;
    let response = await fetch(url);
    let data = await response.json();
    return data.rates;
}

/* FROM → TO */
async function convertFrom() {
    let amount = parseFloat(amountFrom.value);
    if (isNaN(amount)) return;

    let rates = await getRate(fromCurr.value);
    let rate = rates[toCurr.value];

    amountTo.value = (amount * rate).toFixed(4);
    result.innerHTML = `<small>1 ${fromCurr.value} = ${rate.toFixed(4)} ${toCurr.value}</small>`;
}

/* TO → FROM */
async function convertTo() {
    let amount = parseFloat(amountTo.value);
    if (isNaN(amount)) return;

    let rates = await getRate(toCurr.value);
    let rate = rates[fromCurr.value];

    amountFrom.value = (amount * rate).toFixed(4);
    result.innerHTML = `<small>1 ${toCurr.value} = ${rate.toFixed(4)} ${fromCurr.value}</small>`;
}

/* SWAP CURRENCIES + VALUES */
function swapCurrencies() {
    let tempC = fromCurr.value;
    fromCurr.value = toCurr.value;
    toCurr.value = tempC;

    updateFlagFrom();
    updateFlagTo();

    let tempA = amountFrom.value;
    amountFrom.value = amountTo.value;
    amountTo.value = tempA;

    convertFrom();
}

/* FIXED CLOSE FUNCTION */
function closeWindow(){
    document.getElementById("appWindow").style.display = "none";
}

/* SLIDE MENU */
hamburger.onclick = () => menuPanel.classList.toggle("show");

/* MODALS */
function openHelp(){ helpModal.style.display="block"; }
function closeHelp(){ helpModal.style.display="none"; }
function openAbout(){ aboutModal.style.display="block"; }
function closeAbout(){ aboutModal.style.display="none"; }

/* DRAGGABLE WINDOW */
let dragging = false, offX = 0, offY = 0;
let app = document.getElementById("appWindow");

titleBar.onmousedown = (e) => {
    dragging = true;
    offX = e.clientX - app.offsetLeft;
    offY = e.clientY - app.offsetTop;
};

document.onmouseup = () => dragging = false;

document.onmousemove = (e) => {
    if (dragging) {
        app.style.left = (e.clientX - offX) + "px";
        app.style.top  = (e.clientY - offY) + "px";
    }
};
</script>

</body>
</html>
""")


In [24]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
<title>Currency Converter App</title>

<style>
body {
    font-family: Arial, sans-serif;
    background: url('https://images.unsplash.com/photo-1567427017947-545c5f8d16ad?w=1600')
        no-repeat center center fixed;
    background-size: cover;
    color: white;
    display: flex;
    justify-content: center;
    padding-top: 40px;
    overflow: hidden;
}

/* MAIN WINDOW */
#appWindow {
    width: 480px;
    background: rgba(0,0,0,0.75);
    border-radius: 12px;
    box-shadow: 0 0 18px black;
    overflow: hidden;
    position: absolute;
    top: 140px;
    left: 200px;
    backdrop-filter: blur(6px);
    z-index: 10;
}

/* TITLE BAR (DRAGGABLE) */
#titleBar {
    background: rgba(40,40,40,0.85);
    padding: 10px;
    display: flex;
    justify-content: space-between;
    align-items: center;
    cursor: move;
}

#hamburger {
    font-size: 22px;
    cursor: pointer;
}

#closeBtn {
    cursor: pointer;
    font-size: 20px;
    padding: 4px 12px;
    border-radius: 5px;
    background: #666;
}
#closeBtn:hover { background: #888; }

/* SLIDE MENU */
#menuPanel {
    position: absolute;
    width: 200px;
    background: rgba(0,0,0,0.88);
    left: -240px;
    top: 60px;
    padding: 12px;
    border-radius: 10px;
    transition: left 0.35s ease;
    box-shadow: 0 0 14px black;
    z-index: 999;
}

#menuPanel.show {
    left: 15px;
}

#menuPanel div {
    padding: 12px;
    cursor: pointer;
}
#menuPanel div:hover { background: #444; }

/* MODALS (INSIDE APP WINDOW) */
.modal {
    display: none;
    position: absolute;
    top: 50%; left: 50%;
    transform: translate(-50%, -50%);
    background: rgba(0,0,0,0.92);
    padding: 20px;
    width: 330px;
    border-radius: 10px;
    box-shadow: 0 0 20px black;
    z-index: 2000;
}

.modal button {
    margin-top: 15px;
    padding: 10px 18px;
    border-radius: 6px;
    background: #00aaff;
    border: none;
    cursor: pointer;
}

/* MAIN CONTENT */
#box {
    padding: 20px;
    text-align: center;
}

.inputRow {
    display: flex;
    justify-content: center;
    align-items: center;
    margin-top: 12px;
}

input.amountField {
    width: 50%;
    padding: 12px;
    font-size: 18px;
    border-radius: 6px;
    border: none;
    margin: 8px;
}

select {
    padding: 10px;
    font-size: 16px;
    border-radius: 6px;
    margin-left: 10px;
}

.flag {
    font-size: 28px;
    margin-right: 10px;
}

/* VERTICAL SWAP BUTTON */
#swapBtn {
    font-size: 35px;
    cursor: pointer;
    margin: 0 10px;
    user-select: none;
}
#swapBtn:hover {
    color: #00aaff;
    transform: scale(1.2);
}

/* CLEAN RATE DISPLAY */
#result {
    margin-top: 20px;
    font-size: 18px;
}
</style>
</head>

<body>

<!-- SLIDE MENU -->
<div id="menuPanel">
    <div onclick="openHelp()">Help</div>
    <div onclick="openAbout()">About</div>
</div>

<!-- HELP MODAL -->
<div id="helpModal" class="modal">
    <h3>Help</h3>
    <p>
        You may type in either field.<br>
        FROM updates TO.<br>
        TO updates FROM.<br><br>
        ⇅ swaps currencies instantly.
    </p>
    <button onclick="closeHelp()">Close</button>
</div>

<!-- ABOUT MODAL -->
<div id="aboutModal" class="modal">
    <h3>About</h3>
    <p>
        Live two-way currency converter with draggable UI,<br>
        animated menu, euro background and real-time API.
    </p>
    <button onclick="closeAbout()">Close</button>
</div>

<!-- MAIN APP WINDOW -->
<div id="appWindow">

    <div id="titleBar">
        <div style="display:flex;align-items:center;">
            <span id="hamburger">☰</span>
            <span style="margin-left:10px;font-weight:bold;">Currency Converter</span>
        </div>
        <div id="closeBtn" onclick="closeWindow()">✕</div>
    </div>

    <div id="box">

        <h2>Live Currency Converter</h2>

        <!-- FROM FIELD -->
        <div class="inputRow">
            <span class="flag" id="flagFrom">🇺🇸</span>
            <input class="amountField" id="amountFrom" type="number" placeholder="Amount"
                   oninput="convertFrom()">
            <select id="fromCurr" onchange="updateFlagFrom(); convertFrom();">
                <option value="USD">USD</option>
                <option value="EUR">EUR</option>
                <option value="GBP">GBP</option>
                <option value="INR">INR</option>
            </select>
        </div>

        <!-- SWAP BUTTON -->
        <div style="text-align:center;margin:5px;">
            <span id="swapBtn" onclick="swapCurrencies()">⇅</span>
        </div>

        <!-- TO FIELD -->
        <div class="inputRow">
            <span class="flag" id="flagTo">🇪🇺</span>
            <input class="amountField" id="amountTo" type="number" placeholder="Amount"
                   oninput="convertTo()">
            <select id="toCurr" onchange="updateFlagTo(); convertFrom();">
                <option value="USD">USD</option>
                <option value="EUR" selected>EUR</option>
                <option value="GBP">GBP</option>
                <option value="INR">INR</option>
            </select>
        </div>

        <div id="result"></div>

    </div>
</div>

<script>
/* FLAGS */
const flags = { USD:"🇺🇸", EUR:"🇪🇺", GBP:"🇬🇧", INR:"🇮🇳" };

function updateFlagFrom(){ flagFrom.textContent = flags[fromCurr.value]; }
function updateFlagTo(){ flagTo.textContent = flags[toCurr.value]; }

/* LIVE RATES API */
async function getRate(base) {
    let url = `https://open.er-api.com/v6/latest/${base}`;
    let response = await fetch(url);
    let data = await response.json();
    return data.rates;
}

/* FROM → TO */
async function convertFrom() {
    let amount = parseFloat(amountFrom.value);
    if (isNaN(amount)) return;

    let rates = await getRate(fromCurr.value);
    let rate = rates[toCurr.value];

    amountTo.value = (amount * rate).toFixed(4);
    result.innerHTML = `<small>1 ${fromCurr.value} = ${rate.toFixed(4)} ${toCurr.value}</small>`;
}

/* TO → FROM */
async function convertTo() {
    let amount = parseFloat(amountTo.value);
    if (isNaN(amount)) return;

    let rates = await getRate(toCurr.value);
    let rate = rates[fromCurr.value];

    amountFrom.value = (amount * rate).toFixed(4);
    result.innerHTML = `<small>1 ${toCurr.value} = ${rate.toFixed(4)} ${fromCurr.value}</small>`;
}

/* SWAP CURRENCIES + VALUES */
function swapCurrencies() {
    let tempC = fromCurr.value;
    fromCurr.value = toCurr.value;
    toCurr.value = tempC;

    updateFlagFrom();
    updateFlagTo();

    let tempA = amountFrom.value;
    amountFrom.value = amountTo.value;
    amountTo.value = tempA;

    convertFrom();
}

/* FIXED CLOSE FUNCTION */
function closeWindow(){
    document.getElementById("appWindow").style.display = "none";
}

/* SLIDE MENU */
hamburger.onclick = () => menuPanel.classList.toggle("show");

/* MODALS */
function openHelp(){ helpModal.style.display="block"; }
function closeHelp(){ helpModal.style.display="none"; }
function openAbout(){ aboutModal.style.display="block"; }
function closeAbout(){ aboutModal.style.display="none"; }

/* DRAGGABLE WINDOW */
let dragging = false, offX = 0, offY = 0;
let app = document.getElementById("appWindow");

titleBar.onmousedown = (e) => {
    dragging = true;
    offX = e.clientX - app.offsetLeft;
    offY = e.clientY - app.offsetTop;
};

document.onmouseup = () => dragging = false;

document.onmousemove = (e) => {
    if (dragging) {
        app.style.left = (e.clientX - offX) + "px";
        app.style.top  = (e.clientY - offY) + "px";
    }
};
</script>

</body>
</html>
""")
